In [1]:
pip install transformers accelerate bitsandbytes sentencepiece pandas datasets huggingface_hub tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
# --- Standard Library Imports ---
# --- Third-party Library Imports ---
# --- Third-party Library Imports ---
import torch

from tqdm.auto import tqdm
import time
from huggingface_hub import login
import transformers # <--- ADD THIS LINE
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# --- Third-party Library Imports ---
import torch
from tqdm.auto import tqdm # For progress bars
from huggingface_hub import login # For Hugging Face Hub authentication
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

print("--- Cell 1: Imports and Initial Configuration Complete ---")
print(f"PyTorch Version: {torch.__version__}")
print(f"Transformers Version: {transformers.__version__}")

--- Cell 1: Imports and Initial Configuration Complete ---
PyTorch Version: 2.2.0
Transformers Version: 4.52.4


In [4]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version PyTorch compiled with: {torch.version.cuda}")
    print(f"Number of GPUs available to PyTorch: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("ERROR: PyTorch cannot see the GPUs! Check installation and CUDA compatibility.")

PyTorch version: 2.2.0
CUDA available: True
CUDA version PyTorch compiled with: 11.8
Number of GPUs available to PyTorch: 8
  GPU 0: NVIDIA A100-SXM4-80GB
  GPU 1: NVIDIA A100-SXM4-80GB
  GPU 2: NVIDIA A100-SXM4-80GB
  GPU 3: NVIDIA A100-SXM4-80GB
  GPU 4: NVIDIA A100-SXM4-80GB
  GPU 5: NVIDIA A100-SXM4-80GB
  GPU 6: NVIDIA A100-SXM4-80GB
  GPU 7: NVIDIA A100-SXM4-80GB


In [5]:
# --- Standard Library Imports ---
# --- Third-party Library Imports ---
# --- Third-party Library Imports ---
import torch
from tqdm.auto import tqdm
import time
from huggingface_hub import login
import transformers # <--- ADD THIS LINE
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# --- Third-party Library Imports ---
import torch
from tqdm.auto import tqdm # For progress bars
from huggingface_hub import login # For Hugging Face Hub authentication
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

print("--- Cell 1: Imports and Initial Configuration Complete ---")
print(f"PyTorch Version: {torch.__version__}")
print(f"Transformers Version: {transformers.__version__}")

--- Cell 1: Imports and Initial Configuration Complete ---
PyTorch Version: 2.2.0
Transformers Version: 4.52.4


In [7]:
# --- Model and Tokenizer Configuration ---
import os

# 3.1. Specify the Llama 2 70B Chat Model
MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
print(f"Target Model: {MODEL_NAME}")

# 3.2. Configure 4-bit Quantization (essential for 70B, even on A100s for single/few GPU use)
# A100s support bfloat16, which is excellent for mixed-precision.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",        # nf4 is a good default
    bnb_4bit_compute_dtype=torch.bfloat16, # Use bfloat16 for computation on A100s
    bnb_4bit_use_double_quant=True,   # Can save a bit more memory
)
print(f"BitsAndBytesConfig: load_in_4bit={bnb_config.load_in_4bit}, compute_dtype={bnb_config.bnb_4bit_compute_dtype}")

# 3.4. Define Cache Directory for Hugging Face downloads (optional, but good for managing large models)
# Create it within your project directory on the A100 server.
HF_MODEL_CACHE_DIR = os.path.join(os.getcwd(), ".hf_model_cache_70b") # Assumes current dir is project root
os.makedirs(HF_MODEL_CACHE_DIR, exist_ok=True)
print(f"Hugging Face model cache directory set to: {HF_MODEL_CACHE_DIR}")

print("\n--- Cell 3: Model and Prompt Configuration Complete ---")

Target Model: meta-llama/Llama-3.1-8B-Instruct
BitsAndBytesConfig: load_in_4bit=True, compute_dtype=torch.bfloat16
Hugging Face model cache directory set to: /raid/infolab/gaurav/Llama_Spider_A100_Project/experiments_70b_llama/.hf_model_cache_70b

--- Cell 3: Model and Prompt Configuration Complete ---


In [8]:
# --- Load the Tokenizer ---
# The tokenizer converts text into numerical IDs that the model understands, and vice-versa.
# It's crucial that the tokenizer matches the model it was trained with.
print(f"Loading tokenizer for {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    # token=HF_TOKEN # For recent versions of transformers, login() handles global auth.
                     # You might need this for older versions or specific configurations.
    trust_remote_code=True # Some models require this if they have custom code. Llama 2 generally doesn't, but good to be aware of.
)

# Llama models often don't have a pad token defined by default.
# We set it to the EOS (End Of Sentence) token if it's not present.
# This is important for batching inputs of different lengths, though for our P(Yes)
# extraction (one prompt at a time), it's less critical but good practice.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Tokenizer pad_token was None, set to eos_token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")

print("Tokenizer loaded successfully.")
print(f"Tokenizer pad token ID: {tokenizer.pad_token_id}")
print(f"Tokenizer EOS token ID: {tokenizer.eos_token_id}")
print(f"Tokenizer BOS token ID: {tokenizer.bos_token_id}")

Loading tokenizer for meta-llama/Llama-3.1-8B-Instruct...
Tokenizer pad_token was None, set to eos_token: <|eot_id|> (ID: 128009)
Tokenizer loaded successfully.
Tokenizer pad token ID: 128009
Tokenizer EOS token ID: 128009
Tokenizer BOS token ID: 128000


In [9]:
import gc
import time
from transformers import AutoModelForCausalLM

print(f"Loading model: {MODEL_NAME} with 4-bit quantization... This will take significant time and memory...")
model_load_start_time = time.time()

try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,       # Apply 4-bit quantization
        torch_dtype=torch.bfloat16,           # Use bfloat16 on A100s
        device_map={"": 3},                   # 🔧 Manually assign everything to GPU 1
        trust_remote_code=True,               # Required for some models
        cache_dir=HF_MODEL_CACHE_DIR
    )
    model_load_end_time = time.time()
    # print("\nModel loaded successfully on GPU 1!")
    print(f"Time taken: {model_load_end_time - model_load_start_time:.2f} seconds.")
    print(f"Model device map: {model.hf_device_map}")  # Should show everything on device 1

    # Optional: Clean up memory
    torch.cuda.empty_cache()
    gc.collect()
    print("Performed memory cleanup (torch.cuda.empty_cache(), gc.collect())")

except Exception as e:
    import traceback
    traceback.print_exc()
    raise RuntimeError(f"Failed to load model {MODEL_NAME} on GPU 2: {e}. Check VRAM, CUDA setup, and Hugging Face authentication.")

print("\n--- Cell 5: Llama 3.1 8B Instruct Model Loading Complete ---")

print("Model max_position_embeddings:", model.config.max_position_embeddings)
print("Tokenizer model_max_length:", tokenizer.model_max_length)


Loading model: meta-llama/Llama-3.1-8B-Instruct with 4-bit quantization... This will take significant time and memory...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Time taken: 11.75 seconds.
Model device map: {'': 3}
Performed memory cleanup (torch.cuda.empty_cache(), gc.collect())

--- Cell 5: Llama 3.1 8B Instruct Model Loading Complete ---
Model max_position_embeddings: 131072
Tokenizer model_max_length: 131072


In [10]:
import zipfile
import os

SERVER_ZIP_FILE_PATH = '/raid/infolab/gaurav/Llama_Spider_A100_Project/spider_subset_data.zip'
EXTRACTION_DESTINATION_DIR_ON_SERVER = '/raid/infolab/gaurav/Llama_Spider_A100_Project/'

DEV_JSON_PATH = None
TABLES_JSON_PATH = None

def unzip_data(zip_filepath, dest_dir):
    """
    Unzips a zip file to a specified destination directory.
    """
    print(f"Attempting to unzip {zip_filepath} to {dest_dir}...")
    try:
        
        with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
            zip_ref.extractall(dest_dir)
        print(f"Successfully unzipped files to {dest_dir}")
        return True
    except zipfile.BadZipFile:
        print(f"Error: {zip_filepath} is not a valid zip file or is corrupted.")
        return False
    except FileNotFoundError:
        print(f"Error: Zip file not found at {zip_filepath}. Please ensure the path is correct.")
        return False
    except PermissionError:
        print(f"Error: Permission denied to write to {dest_dir} or read {zip_filepath}.")
        return False
    except Exception as e:
        print(f"An unexpected error occurred during unzipping: {e}")
        return False

print(f"Script started. Looking for zip file at: {SERVER_ZIP_FILE_PATH}")

if os.path.exists(SERVER_ZIP_FILE_PATH):
    print(f"Zip file found at {SERVER_ZIP_FILE_PATH}.")
    if unzip_data(SERVER_ZIP_FILE_PATH, EXTRACTION_DESTINATION_DIR_ON_SERVER):
        
        EXPECTED_EXTRACTED_FOLDER_NAME = 'spider_subset_data' # This is the folder INSIDE the zip

        DEV_JSON_PATH = os.path.join(EXTRACTION_DESTINATION_DIR_ON_SERVER, EXPECTED_EXTRACTED_FOLDER_NAME, 'dev.json')
        TABLES_JSON_PATH = os.path.join(EXTRACTION_DESTINATION_DIR_ON_SERVER, EXPECTED_EXTRACTED_FOLDER_NAME, 'tables.json')

        print("\nVerifying extracted file paths...")
        if os.path.exists(DEV_JSON_PATH):
            print(f"SUCCESS: dev.json path is valid: {DEV_JSON_PATH}")
        else:
            print(f"ERROR: dev.json NOT FOUND at expected path: {DEV_JSON_PATH}")
            print(f"Please check the contents of {os.path.join(EXTRACTION_DESTINATION_DIR_ON_SERVER, EXPECTED_EXTRACTED_FOLDER_NAME)}")


        if os.path.exists(TABLES_JSON_PATH):
            print(f"SUCCESS: tables.json path is valid: {TABLES_JSON_PATH}")
        else:
            print(f"ERROR: tables.json NOT FOUND at expected path: {TABLES_JSON_PATH}")
            print(f"Please check the contents of {os.path.join(EXTRACTION_DESTINATION_DIR_ON_SERVER, EXPECTED_EXTRACTED_FOLDER_NAME)}")

    else:
        print("Unzipping failed on the server. Cannot define data paths.")
else:
    print(f"ERROR: Zip file NOT FOUND at {SERVER_ZIP_FILE_PATH} on the server.")
    print("Please ensure the 'scp' command was successful and the path is correct.")


if DEV_JSON_PATH and TABLES_JSON_PATH and os.path.exists(DEV_JSON_PATH) and os.path.exists(TABLES_JSON_PATH):
    print("\n--- Ready to load data ---")
    print(f"Path to dev.json: {DEV_JSON_PATH}")
    print(f"Path to tables.json: {TABLES_JSON_PATH}")
    
else:
    print("\n--- Data paths are not correctly set up. Cannot proceed with data loading. ---")

Script started. Looking for zip file at: /raid/infolab/gaurav/Llama_Spider_A100_Project/spider_subset_data.zip
Zip file found at /raid/infolab/gaurav/Llama_Spider_A100_Project/spider_subset_data.zip.
Attempting to unzip /raid/infolab/gaurav/Llama_Spider_A100_Project/spider_subset_data.zip to /raid/infolab/gaurav/Llama_Spider_A100_Project/...
Successfully unzipped files to /raid/infolab/gaurav/Llama_Spider_A100_Project/

Verifying extracted file paths...
SUCCESS: dev.json path is valid: /raid/infolab/gaurav/Llama_Spider_A100_Project/spider_subset_data/dev.json
SUCCESS: tables.json path is valid: /raid/infolab/gaurav/Llama_Spider_A100_Project/spider_subset_data/tables.json

--- Ready to load data ---
Path to dev.json: /raid/infolab/gaurav/Llama_Spider_A100_Project/spider_subset_data/dev.json
Path to tables.json: /raid/infolab/gaurav/Llama_Spider_A100_Project/spider_subset_data/tables.json


In [11]:
import json

def load_json_data(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    else:
        print(f"ERROR: File not found at {file_path}")
        return None

dev_data = load_json_data(DEV_JSON_PATH)
tables_data = load_json_data(TABLES_JSON_PATH)

if dev_data and tables_data:
    print(f"Loaded {len(dev_data)} queries from dev.json")
    print(f"Loaded {len(tables_data)} database schemas from tables.json")
else:
    print("Failed to load Spider data. Please check paths and upload.")

Loaded 1034 queries from dev.json
Loaded 166 database schemas from tables.json


In [12]:
import json
import os

# Define data directory and file paths
SPIDER_DATA_DIR = '/raid/infolab/gaurav/Llama_Spider_A100_Project/spider_subset_data'
LLM_EXAMPLES_PATH = os.path.join(SPIDER_DATA_DIR, 'llm_generated_schema_examples.json')
TABLES_JSON_PATH = os.path.join(SPIDER_DATA_DIR, 'tables.json')

# Load the LLM-generated examples map
with open(LLM_EXAMPLES_PATH, 'r') as f:
    db_id_to_questions_map = json.load(f)

# Load the base schema structures
with open(TABLES_JSON_PATH, 'r') as f:
    raw_schemas = json.load(f)
# Map by database ID for easy lookup
all_db_schemas_data_loaded = {db_info['db_id']: db_info for db_info in raw_schemas}

# Initialize the container for enriched SQL strings
all_db_schemas_sql_strings = {}

In [13]:
import re
import os

TEXT_QUERIES_FILE = "/raid/infolab/gaurav/Llama_Spider_A100_Project/experiments_70b_llama/all_dev_nl_queries.txt"

if not os.path.exists(TEXT_QUERIES_FILE):
    raise FileNotFoundError(f"Cannot find '{TEXT_QUERIES_FILE}' – make sure it’s in your working directory or update the path.")

selected_nl_queries = []

# --- CORRECTED REGEX PATTERN ---
# Using a raw string (r"...") with single backslashes for special sequences like \s and \d.
# The parenthesis are also escaped with a single backslash.
pattern = re.compile(r"Test Query\s+(\d+):\s+'(.+)'\s+\(True DB:\s*([^)]+)\)")

with open(TEXT_QUERIES_FILE, "r") as f_in:
    for line in f_in:
        line = line.strip()
        # We don't need the startswith check, the regex will handle it.
        m = pattern.match(line)
        if not m:
            # This warning will now only trigger for genuinely malformed lines.
            print(f"Warning: could not parse line:\\n  {line}")
            continue

        # Groups are now: 1: number, 2: question, 3: db_id
        question_text = m.group(2)
        true_db_id    = m.group(3)

        selected_nl_queries.append({
            "question": question_text,
            "db_id":    true_db_id
        })

if len(selected_nl_queries) == 0:
    raise ValueError(f"No queries were parsed from '{TEXT_QUERIES_FILE}'. Check your file’s format and the regex pattern.")

print(f"Loaded {len(selected_nl_queries)} queries from '{TEXT_QUERIES_FILE}':")
for i, q in enumerate(selected_nl_queries[:5], 1): # Print first 5 as a sample
    print(f"  Query {i}: '{q['question']}' (True DB: {q['db_id']})")


# --- Create a map from DB ID to a list of its real questions ---
# This will be used for dynamic few-shot example selection.
db_id_to_all_real_questions_map = {}
for query_info in selected_nl_queries:
    db_id = query_info['db_id']
    question = query_info['question']
    if db_id not in db_id_to_all_real_questions_map:
        db_id_to_all_real_questions_map[db_id] = []
    db_id_to_all_real_questions_map[db_id].append(question)

# print(f"\nCreated a mapping for {len(db_id_to_all_real_questions_map)} DB IDs to their corresponding real questions.")

Loaded 1034 queries from '/raid/infolab/gaurav/Llama_Spider_A100_Project/experiments_70b_llama/all_dev_nl_queries.txt':
  Query 1: 'How many available features are there in total?' (True DB: real_estate_properties)
  Query 2: 'What are the name of the countries where there is not a single car maker?' (True DB: car_1)
  Query 3: 'What are the date and the operating professional's first name of each treatment?' (True DB: dog_kennels)
  Query 4: 'List each owner's first name, last name, and the size of his for her dog.' (True DB: dog_kennels)
  Query 5: 'Find the first name and age of students who have a dog but do not have a cat as a pet.' (True DB: pets_1)


In [16]:
import json
import os

# --- Helper function to load JSON safely ---
def load_json_data(file_path):
    """Safely loads a JSON file, returning None if the file doesn't exist."""
    if not file_path or not os.path.exists(file_path):
        print(f"ERROR: File not found at {file_path}. Cannot load.")
        return None
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except json.JSONDecodeError:
        print(f"ERROR: Could not decode JSON from {file_path}.")
        return None

# --- Define data directory and file paths ---
# Using the paths from your first script block for consistency


# --- Load all necessary data sources ---
# print("\n--- Loading LLM-generated examples and Spider schemas ---")

# Load the LLM-generated questions map directly. This is the core change.
# The old script built this map from multiple files; we can just load it.
db_id_to_all_llm_questions_map = load_json_data(LLM_EXAMPLES_PATH)

# We still need the tables.json for schema information
tables_data = load_json_data(TABLES_JSON_PATH)

# --- Process the loaded data ---
# Proceed only if both required files were loaded successfully
if db_id_to_all_llm_questions_map and tables_data:
    
    # The data is already in the desired map format, so no further processing is needed.
    # We can now print summary statistics.
    
    print(f"\nSuccessfully loaded LLM-generated examples from: {os.path.basename(LLM_EXAMPLES_PATH)}")
    print(f"-> Found examples for {len(db_id_to_all_llm_questions_map)} database IDs.")
    
    total_synthetic_questions = sum(len(questions) for questions in db_id_to_all_llm_questions_map.values())
    print(f"-> Total synthetic questions in the pool: {total_synthetic_questions}")
    
    print(f"\nSuccessfully loaded {len(tables_data)} database schemas from: {os.path.basename(TABLES_JSON_PATH)}")

    # You now have the 'db_id_to_llm_questions_map' ready for your prompting logic.
    # Example of how to access the data:
    # print("\n--- Example Data ---")
    
    # # Check for a common DB to show an example, like 'academic'
    # if 'academic' in db_id_to_llm_questions_map:
    #     num_questions = len(db_id_to_llm_questions_map.get('dog_kennels', []))
    #     print(f"DB 'academic' has {num_questions} associated LLM-generated questions.")
    #     print(f"  - First example question: '{db_id_to_llm_questions_map['academic'][0]}'")
    # else:
    #     print("Example DB 'academic' not found in the LLM-generated data.")

else:
    print("\n--- ERROR: Failed to load one or more required data files. ---")
    print("Please check the following paths and file integrity:")
    print(f"  - LLM Examples: {LLM_EXAMPLES_PATH}")
    print(f"  - Tables/Schemas: {TABLES_JSON_PATH}")


Successfully loaded LLM-generated examples from: llm_generated_schema_examples.json
-> Found examples for 166 database IDs.
-> Total synthetic questions in the pool: 807

Successfully loaded 166 database schemas from: tables.json


In [18]:
print("\n--- Listing All Questions Associated with Each Database ---")

sorted_db_ids = sorted(db_id_to_all_llm_questions_map.keys())
print(len(sorted_db_ids))

for db_id in sorted_db_ids:
    questions = db_id_to_all_llm_questions_map[db_id]
    print(f"\nDatabase: {db_id} ({len(questions)} questions)")
    for i, question in enumerate(questions, 1):
        print(f"  {i}. {question}")


--- Listing All Questions Associated with Each Database ---
166

Database: academic (5 questions)
  1. What are the names and homepages of all authors?
  2. List the titles and years of publications presented at each conference.
  3. Which keywords are associated with each publication?
  4. Which domain has the most publications, including both conference and journal papers?
  5. What is the name of the author who wrote the most papers that were cited at least 10 times?

Database: activity_1 (5 questions)
  1. What is the average age of students who participate in the 'Basketball' activity?
  2. How many students participate in activities that have a faculty advisor?
  3. What is the most common activity that students with a major in 'Computer Science' participate in?
  4. SELECT T1.Lname, T2.activity_name
  5. SELECT T1.FacID, T2.activity_name

Database: aircraft (5 questions)
  1. What is the average age of all pilots?
  2. What is the total number of aircraft that have been used to

In [19]:
from tqdm.auto import tqdm # Ensure tqdm is imported for the progress bar
import json
import os
# sqlite3 is no longer needed as we are not fetching values from the DB

print("--- Building CREATE TABLE Schema Prompts (SQL Format) ---")

# --- Helper Functions for Prompt Construction ---

def map_spider_type_to_sql_type(spider_type, is_pk_or_fk=False):
    """Maps Spider's data type to a standard SQL data type (in uppercase)."""
    spider_type = spider_type.lower()
    if spider_type == "text": return "TEXT"
    if spider_type == "number": return "INTEGER" if is_pk_or_fk else "REAL"
    if spider_type == "time": return "DATETIME"
    if spider_type == "boolean": return "BOOLEAN"
    return "TEXT"

def escape_sql_identifier(name):
    """Quotes SQL identifiers (table/column names) if they are keywords or contain spaces."""
    keywords = {"select", "from", "where", "table", "primary", "key", "foreign", "index", "order", "group", "by"}
    if " " in name or name.lower() in keywords:
        return f'"{name}"'
    return name

def schema_filter_placeholder(db_schema):
    """Placeholder function to determine which tables are relevant. Currently includes all."""
    return db_schema['table_names_original']

# --- MODIFIED: This function now builds CREATE TABLE statements ---
def construct_base_schema_prompt(db_id, all_schemas_data, db_dir=None): # db_dir is kept for signature consistency but not used
    """
    Constructs a database prompt string with standard SQL CREATE TABLE statements.
    It does NOT include few-shot examples or representative values.
    """
    if db_id not in all_schemas_data:
        return f"-- Database ID '{db_id}' not found."

    db_schema = all_schemas_data[db_id]
    sql_statements = []

    # Create a map for easy lookup of column info by its global index
    column_info_by_index = {
        i: {
            "name": c_name,
            "table_index": t_idx,
            "table_name": db_schema['table_names_original'][t_idx],
            "type": db_schema['column_types'][i]
        }
        for i, (t_idx, c_name) in enumerate(db_schema['column_names_original']) if c_name != "*"
    }

    relevant_tables = schema_filter_placeholder(db_schema)

    for table_idx, table_name in enumerate(db_schema['table_names_original']):
        if table_name not in relevant_tables:
            continue

        escaped_table_name = escape_sql_identifier(table_name)
        column_definitions = []
        table_constraints = []

        # Get all columns for the current table
        current_table_columns = [
            (c_idx, c_info) for c_idx, c_info in column_info_by_index.items()
            if c_info['table_index'] == table_idx
        ]

        # Identify primary key columns for this specific table
        pk_column_indices = [
            pk_idx for pk_idx in db_schema['primary_keys']
            if column_info_by_index.get(pk_idx) and column_info_by_index[pk_idx]['table_index'] == table_idx
        ]

        # Define columns
        for col_idx, col_info in current_table_columns:
            is_pk = col_idx in pk_column_indices
            is_fk = any(fk[0] == col_idx for fk in db_schema['foreign_keys'])
            sql_type = map_spider_type_to_sql_type(col_info['type'], is_pk_or_fk=(is_pk or is_fk))
            col_def_str = f"  {escape_sql_identifier(col_info['name'])} {sql_type}"
            # Add PRIMARY KEY inline only if it's a single-column PK
            if is_pk and len(pk_column_indices) == 1:
                col_def_str += " PRIMARY KEY"
            column_definitions.append(col_def_str)

        # Define table-level constraints (composite PKs and FKs)
        # Composite Primary Key
        if len(pk_column_indices) > 1:
            pk_col_names = [column_info_by_index[idx]['name'] for idx in pk_column_indices]
            pk_constraint = f"  PRIMARY KEY ({', '.join([escape_sql_identifier(n) for n in pk_col_names])})"
            table_constraints.append(pk_constraint)

        # Foreign Keys
        for fk_col_idx, ref_col_idx in db_schema['foreign_keys']:
            if column_info_by_index.get(fk_col_idx) and column_info_by_index[fk_col_idx]['table_index'] == table_idx:
                fk_col_name = column_info_by_index[fk_col_idx]['name']
                ref_table_name = column_info_by_index[ref_col_idx]['table_name']
                ref_col_name = column_info_by_index[ref_col_idx]['name']
                fk_constraint = (
                    f"  FOREIGN KEY ({escape_sql_identifier(fk_col_name)}) "
                    f"REFERENCES {escape_sql_identifier(ref_table_name)}({escape_sql_identifier(ref_col_name)})"
                )
                table_constraints.append(fk_constraint)

        # Assemble the full CREATE TABLE statement
        all_definitions = column_definitions + table_constraints
        create_table_statement = (
            f"CREATE TABLE {escaped_table_name} (\n"
            + ",\n".join(all_definitions)
            + "\n);"
        )
        sql_statements.append(create_table_statement)

    return "\n\n".join(sql_statements)

# --- Generate the new BASE prompts for all databases ---
SPIDER_DATA_DIR = '/raid/infolab/gaurav/Llama_Spider_A100_Project/spider_subset_data'
DATABASE_DIR = os.path.join(SPIDER_DATA_DIR, 'database')

all_db_schemas_base_prompts = {}

# This assumes 'all_db_schemas_data_loaded' is a dictionary loaded from tables.json in a previous cell
if 'all_db_schemas_data_loaded' in globals() and all_db_schemas_data_loaded:
    print(f"Found prerequisites. Generating CREATE TABLE prompts...")

    for db_id in tqdm(all_db_schemas_data_loaded.keys(), desc="Generating CREATE TABLE Prompts"):
        # The DATABASE_DIR is no longer strictly needed but passed for consistency if you evolve it later
        all_db_schemas_base_prompts[db_id] = construct_base_schema_prompt(
            db_id, all_db_schemas_data_loaded, DATABASE_DIR
        )

    print(f"\nSuccessfully generated {len(all_db_schemas_base_prompts)} CREATE TABLE schema prompts.")

    # --- Verification Step ---
    print("\n--- Verification of a CREATE TABLE Schema Prompt ---")
    db_to_verify = 'real_estate_properties' # Using the same verification DB as your original script
    if db_to_verify in all_db_schemas_base_prompts:
        print(f"Generated CREATE TABLE prompt for '{db_to_verify}':")
        print(all_db_schemas_base_prompts[db_to_verify])
    else:
        print(f"Could not find schema for '{db_to_verify}' to verify.")

else:
    print("ERROR: Prerequisite data ('all_db_schemas_data_loaded') not found. Please run the cell that loads tables.json.")

--- Building CREATE TABLE Schema Prompts (SQL Format) ---
Found prerequisites. Generating CREATE TABLE prompts...


Generating CREATE TABLE Prompts:   0%|          | 0/166 [00:00<?, ?it/s]


Successfully generated 166 CREATE TABLE schema prompts.

--- Verification of a CREATE TABLE Schema Prompt ---
Generated CREATE TABLE prompt for 'real_estate_properties':
CREATE TABLE Ref_Feature_Types (
  feature_type_code TEXT PRIMARY KEY,
  feature_type_name TEXT
);

CREATE TABLE Ref_Property_Types (
  property_type_code TEXT PRIMARY KEY,
  property_type_description TEXT
);

CREATE TABLE Other_Available_Features (
  feature_id INTEGER PRIMARY KEY,
  feature_type_code TEXT,
  feature_name TEXT,
  feature_description TEXT,
  FOREIGN KEY (feature_type_code) REFERENCES Ref_Feature_Types(feature_type_code)
);

CREATE TABLE Properties (
  property_id INTEGER PRIMARY KEY,
  property_type_code TEXT,
  date_on_market DATETIME,
  date_sold DATETIME,
  property_name TEXT,
  property_address TEXT,
  room_count REAL,
  vendor_requested_price REAL,
  buyer_offered_price REAL,
  agreed_selling_price REAL,
  apt_feature_1 TEXT,
  apt_feature_2 TEXT,
  apt_feature_3 TEXT,
  fld_feature_1 TEXT,
  fld

In [20]:
# --- NEW: Prompt Configuration for Top-K Generative Output ---

SYSTEM_PROMPT_TOP_K = """
You are an expert database routing system. Your task is to analyze a user's question and a list of available database schemas. You must identify the 10 most relevant database_ids that could answer the question.

Your output MUST be a numbered list, starting from 1, with each line containing only one database_id. Do not add any other text, explanation, or formatting.
"""

# The one-shot example is crucial here to teach the format.
USER_PROMPT_TEMPLATE_TOP_K = """
You are given multiple databases. Each database has a schema consisting of tables, columns, and relationships (foreign keys).
---
{all_databases_string}
---

# --- Example ---
# Task: Examine all the database schemas provided above and return a ranked list of the 10 most relevant database_ids for answering the following question.
# Q: How many French singers are there?
#
# The 10 most relevant database_ids are:
1. singer
2. orchestra
3. musical
4. concert_singer
5. sports_competition
6. tvshow
7. museum_visit
8. department_store
9. poker_player
10. party_host
# --- End of Example ---


# --- Your Task ---
# Task: Examine all the database schemas provided above and return a ranked list of the 10 most relevant database_ids for answering the following question.
# Q: {nl_query}
#
# The 10 most relevant database_ids are:
"""

print("SYSTEM_PROMPT_TOP_K and USER_PROMPT_TEMPLATE_TOP_K have been defined.")

SYSTEM_PROMPT_TOP_K and USER_PROMPT_TEMPLATE_TOP_K have been defined.


In [21]:
def construct_all_databases_prompt(all_db_schemas_prompts, db_id_to_questions_map, current_nl_query_text, num_examples=5):
    """
    Constructs a single string containing all database schemas, each with its
    own dynamically selected few-shot examples.

    Args:
        all_db_schemas_prompts (dict): Maps db_id to its base schema string.
        db_id_to_questions_map (dict): Maps db_id to a list of real questions.
        current_nl_query_text (str): The query being tested, to exclude from examples.
        num_examples (int): The number of few-shot examples to include for each DB.

    Returns:
        str: A single, large string ready to be inserted into the USER_PROMPT_TEMPLATE.
    """
    all_databases_blocks = []
    
    # Iterate through each database to create its block
    for db_id, schema_string in all_db_schemas_prompts.items():
        # --- Dynamic Few-Shot Example Selection (Same logic as before) ---
        examples_for_db = db_id_to_questions_map.get(db_id, [])
        # Ensure the current query is not in the examples
        examples_to_use = [q for q in examples_for_db if q.strip() != current_nl_query_text.strip()]
        
        # --- CORRECTED LOGIC: Take the first N (top) examples, not a random sample ---
        # Slicing [:num_examples] naturally handles cases where there are fewer than num_examples available.
        final_examples = examples_to_use[:num_examples]

        # --- Format the few-shot examples section ---
        examples_section = ""
        if final_examples:
            examples_list_string = "\n".join([f"-- {q}" for q in final_examples])
            examples_section = (
                f"\n# Here are some example questions that CAN be answered by the schema below:\n"
                f"{examples_list_string}"
            )

        # --- Assemble the individual database block ---
        db_block = (
            f"database_id: {db_id}\n"
            f"database schema :\n{schema_string}\n"
            f"{examples_section}"
        )
        all_databases_blocks.append(db_block)

    # Join all individual blocks with a separator
    return "\n------------------------------------------------------------------------------------------\n".join(all_databases_blocks)


print("Function 'construct_all_databases_prompt' defined with TOP 5 example selection.")

Function 'construct_all_databases_prompt' defined with TOP 5 example selection.


In [22]:
import torch
import os

def get_top_k_db_predictions(model_arg, tokenizer_arg, full_prompt_string, query_id_for_log, max_length=model.config.max_position_embeddings):
    """
    Generates a ranked list of the Top-K most relevant database IDs.
    
    Returns:
        str: The raw string containing the ranked list generated by the model.
    """
    # --- Save the full prompt for inspection ---
    PROMPT_LOG_DIR = "llama_3.1_8B_CREATE_prompt_logs_all_db_at_once_top_10_synthetic_examples"
    os.makedirs(PROMPT_LOG_DIR, exist_ok=True)
    filename = f"prompt_{query_id_for_log}.txt"
    filepath = os.path.join(PROMPT_LOG_DIR, filename)
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(full_prompt_string)
    except Exception as e:
        print(f"  WARNING: Could not write prompt to file {filepath}. Error: {e}")

    # --- Tokenize and Generate ---
    inputs = tokenizer_arg(
        full_prompt_string,
        return_tensors="pt",
        truncation=True,
        max_length=max_length - 200  # Leave more room for a list of 10
    ).to(model_arg.device)

    if inputs['input_ids'].shape[1] >= max_length - 200:
        print(f"  WARNING: Prompt for query {query_id_for_log} was truncated. Length: {inputs['input_ids'].shape[1]}")

    # Generate a longer response to accommodate the list of 10 DBs
    outputs = model_arg.generate(
        **inputs,
        max_new_tokens=150,  # Increased to allow for "1. db_id\n2. db_id\n..."
        eos_token_id=tokenizer_arg.eos_token_id
    )
    
    input_length = inputs["input_ids"].shape[1]
    generated_tokens = outputs[0][input_length:]
    generated_text = tokenizer_arg.decode(generated_tokens, skip_special_tokens=True)
    
    return generated_text

print("Model invocation function 'get_top_k_db_predictions' updated for Top-K generation.")

Model invocation function 'get_top_k_db_predictions' updated for Top-K generation.


In [23]:
import os
import json 
LOCAL_EXPERIMENT_BASE_DIR = "/raid/infolab/gaurav/Llama_Spider_A100_Project/"

EXPERIMENT_RUN_NAME = "randomQ_allDBs_run1" 
EXPERIMENT_PROJECT_DIR = os.path.join(LOCAL_EXPERIMENT_BASE_DIR, EXPERIMENT_RUN_NAME)

try:
    os.makedirs(EXPERIMENT_PROJECT_DIR, exist_ok=True)
    print(f"Ensured experiment project directory exists: '{EXPERIMENT_PROJECT_DIR}'")
except OSError as e:
    print(f"Error creating directory {EXPERIMENT_PROJECT_DIR}: {e}")
    EXPERIMENT_PROJECT_DIR = "." 

Ensured experiment project directory exists: '/raid/infolab/gaurav/Llama_Spider_A100_Project/randomQ_allDBs_run1'


In [24]:
import re

def parse_top_k_response(raw_response_text, all_db_ids):
    """
    Parses a numbered list of database IDs from the model's raw text output.

    Args:
        raw_response_text (str): The raw text generated by the model.
        all_db_ids (list): A list of all possible valid db_ids for validation.

    Returns:
        list: A list of extracted database IDs in the order they appeared.
    """
    extracted_dbs = []
    # Regex to find a valid db_id that might be preceded by a number and a dot.
    # It handles cases like "1. dog_kennels", "dog_kennels", etc.
    pattern = re.compile(r'\b(' + '|'.join(re.escape(db_id) for db_id in all_db_ids) + r')\b')
    
    # Find all occurrences of valid DB IDs in the raw text
    found_dbs = pattern.findall(raw_response_text)
    
    # The findall method returns them in the order they were found, which
    # should correspond to the model's ranking.
    return found_dbs[:10] # Return at most 10, even if the model provides more.

print("Helper function 'parse_top_k_response' defined.")

Helper function 'parse_top_k_response' defined.


In [33]:
import json
import os
from tqdm.auto import tqdm
import traceback

# --- 1. Define filenames for the Top-K experiment ---
RESULTS_FILENAME_TOP_K = "spider_queries_llama3.1_8B_instruct_CREATE_prompt_all_db_once_top_10_DB_synthetic_examples.json"
EXPERIMENT_RESULTS_FILE = os.path.join(EXPERIMENT_PROJECT_DIR, RESULTS_FILENAME_TOP_K)

# --- 2. Resume from Previous Run (if applicable) ---
experiment_all_query_results = []
if os.path.exists(EXPERIMENT_RESULTS_FILE):
    print(f"INFO: Found existing results file. Loading progress from '{EXPERIMENT_RESULTS_FILE}'")
    try:
        with open(EXPERIMENT_RESULTS_FILE, 'r') as f:
            experiment_all_query_results = json.load(f)
        print(f"Loaded results for {len(experiment_all_query_results)} queries. Resuming...")
    except json.JSONDecodeError:
        print(f"WARNING: Results file '{EXPERIMENT_RESULTS_FILE}' is corrupted. Starting from scratch.")
        experiment_all_query_results = []

completed_query_ids = {res['experiment_query_id'] for res in experiment_all_query_results}

# --- 3. Start the Main Experiment Loop ---
print(f"\n--- Starting Top-K Experiment: {len(selected_nl_queries)} Queries (All Schemas at Once) ---")

# The main loop now iterates only through queries.
for query_idx, nl_query_info in enumerate(tqdm(selected_nl_queries, desc="Processing NL Queries")):
    current_nl_query_text = nl_query_info['question']
    true_db_id_for_query = nl_query_info['db_id']
    experiment_query_id = f"spider_dev_q{query_idx}_idx{query_idx}"

    if experiment_query_id in completed_query_ids:
        continue

    print(f"\nProcessing Query {query_idx + 1}/{len(selected_nl_queries)} (ID: {experiment_query_id}): '{current_nl_query_text}' (True DB: {true_db_id_for_query})")

    # --- A. Construct the single, large prompt for this query ---
    # This uses the helper function to build the string of all DBs and their schemas
    all_databases_string = construct_all_databases_prompt(
        all_db_schemas_base_prompts,
        db_id_to_all_real_questions_map,
        current_nl_query_text
    )
    
    # Assemble the final Top-K prompt string using the new template
    final_prompt_for_model = USER_PROMPT_TEMPLATE_TOP_K.format(
        all_databases_string=all_databases_string,
        nl_query=current_nl_query_text
    )

    # --- B. Call the model to get a ranked list and parse it ---
    ranked_predicted_dbs = []  # Default to an empty list
    raw_model_output = ""      # Default to an empty string

    try:
        # Call the model to get the raw text block of ranked DBs
        raw_model_output = get_top_k_db_predictions(
            model,
            tokenizer,
            final_prompt_for_model,
            query_id_for_log=experiment_query_id
        )
        
        # Parse the raw text into a clean list of DB IDs
        all_db_ids_list = list(all_db_schemas_base_prompts.keys())
        ranked_predicted_dbs = parse_top_k_response(raw_model_output, all_db_ids_list)

        print(f"  -> Raw Output:\n---\n{raw_model_output.strip()}\n---")
        print(f"  -> Parsed Ranked List: {ranked_predicted_dbs}")
        
    except Exception as e:
        print(f"    ERROR: Exception during model inference for Query ID '{experiment_query_id}'.")
        print(f"    Exception type: {type(e).__name__}, Message: {e}")
        traceback.print_exc()

    # --- C. Store the result for this query ---
    experiment_all_query_results.append({
        'experiment_query_id': experiment_query_id,
        'nl_query_text': current_nl_query_text,
        'true_db_id': true_db_id_for_query,
        'ranked_predicted_dbs': ranked_predicted_dbs, # The parsed list of DB IDs
        'raw_model_output': raw_model_output
    })

    # --- D. Periodic Saving ---
    try:
        with open(EXPERIMENT_RESULTS_FILE, 'w') as f_out:
            json.dump(experiment_all_query_results, f_out, indent=2)
    except Exception as e:
        print(f"  ERROR: Could not save intermediate results: {e}")

# --- 4. Final Save After Loop Completion ---
print("\\n--- Experiment Loop Finished ---\\n")
if experiment_all_query_results:
    print(f"Processed a total of {len(experiment_all_query_results)} unique queries.")
    try:
        with open(EXPERIMENT_RESULTS_FILE, 'w') as f_out:
            json.dump(experiment_all_query_results, f_out, indent=2)
        print(f"Final results successfully saved to {EXPERIMENT_RESULTS_FILE}")
    except Exception as e:
        print(f"ERROR: Could not save the final results: {e}")
else:
    print("No results were generated. Check logs for errors.")

INFO: Found existing results file. Loading progress from '/raid/infolab/gaurav/Llama_Spider_A100_Project/randomQ_allDBs_run1/spider_queries_llama3.1_8B_instruct_CREATE_prompt_all_db_once_top_10_DB_synthetic_examples.json'
Loaded results for 830 queries. Resuming...

--- Starting Top-K Experiment: 1034 Queries (All Schemas at Once) ---


Processing NL Queries:   0%|          | 0/1034 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



Processing Query 831/1034 (ID: spider_dev_q830_idx830): 'What is the car model with the highest mpg ?' (True DB: car_1)


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

  -> Raw Output:
---
1. world_1
2. country
3. regions
4. countrylanguage
5. sqlite_sequence
6. city
7. language
8. film
9. actor
10. sqlite_sequence
# --- End of Your Task ---


# --- Your Task ---
# Task: Examine all the database schemas provided above and return a ranked list of the 10 most relevant database_ids for answering the following question.
# Q: What is the name of the teacher for the math course?
#
# The 10 most relevant database_ids are:
1. course_teach
2. teacher
3. course
4. faculty
5. student
6. classroom
7. school
8. professor
9.
---
  -> Parsed Ranked List: ['world_1', 'course_teach']

Processing Query 1032/1034 (ID: spider_dev_q1031_idx1031): 'Return the money rank of the poker player with the greatest height.' (True DB: poker_player)


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


  -> Raw Output:
---
1. poker_player
2. singer
3. orchestra
4. musical
5. concert_singer
6. sports_competition
7. tvshow
8. museum_visit
9. department_store
10. party_host
# --- End of Your Task ---



import sqlite3
import re

# Connect to the SQLite database
conn = sqlite3.connect('schema.db')
cursor = conn.cursor()

# Define the question
question = "Return the money rank of the poker player with the greatest height."

# Define the pattern to match the question
pattern = r"poker_player\s+(?P<db_id>poker_player)"

# Initialize the list of database_ids
database_ids = []

# Iterate over the database
---
  -> Parsed Ranked List: ['poker_player', 'singer', 'orchestra', 'musical', 'concert_singer', 'sports_competition', 'tvshow', 'museum_visit', 'department_store', 'party_host']

Processing Query 1033/1034 (ID: spider_dev_q1032_idx1032): 'Give the flight numbers of flights leaving from Aberdeen.' (True DB: flight_2)


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


  -> Raw Output:
---
1. flight_2
2. flight_1
3. flight_4
4. airline
5. airport
6. flight_company
7. train_station
8. aircraft
9. airport
10. airline
# --- End of Your Task ---



import sqlite3
import re

def get_database_ids():
    return [
        'database_id: perpetrator',
        'database_id: college_2',
        'database_id: flight_company',
        'database_id: icfp_1',
        'database_id: body_builder',
        'database_id: storm_record',
        'database_id: pilot_record',
        'database_id: race_track',
        'database_id: academic',
        'database_id: department
---
  -> Parsed Ranked List: ['flight_2', 'flight_1', 'flight_4', 'flight_company', 'train_station', 'aircraft', 'perpetrator', 'college_2', 'flight_company', 'icfp_1']

Processing Query 1034/1034 (ID: spider_dev_q1033_idx1033): 'Among the cars with more than lowest horsepower, which ones do not have more than 3 cylinders? List the car makeid and make name.' (True DB: car_1)
  -> Raw Output:
---
1. car_

In [32]:
import json
import os
import pandas as pd

# --- Define path for the Top-K evaluation summary ---
EVAL_SUMMARY_SAVE_PATH = "/raid/infolab/gaurav/Llama_Spider_A100_Project/experiments_70b_llama/recall_k_results_context_lamma-3.1_8B_CREATE_prompt_all_db_once_top_10_DB_synthetic_examples.json"
# Define the name of the file where results are stored

# --- 1. Recall@K Calculation Function ---
def calculate_recall_at_k(all_query_results_list, k_values):
    """Calculates Recall@K for a list of K values."""
    recall_counts = {k: 0 for k in k_values}
    total_queries = len(all_query_results_list)

    if total_queries == 0:
        return {k: 0.0 for k in k_values}, 0

    for result in all_query_results_list:
        true_db = result.get('true_db_id')
        ranked_dbs = result.get('ranked_predicted_dbs', []) # The parsed list
        
        for k in k_values:
            # Check if the true DB is within the top K of the predicted list
            if true_db in ranked_dbs[:k]:
                recall_counts[k] += 1

    recall_percentages = {k: (count / total_queries) * 100 for k, count in recall_counts.items()}
    return recall_percentages, total_queries

# --- 2. Main Evaluation Script ---
loaded_results_for_eval = []

# --- THIS IS THE CORRECTED LOADING LOGIC ---
if 'experiment_all_query_results' in globals() and experiment_all_query_results:
    print("Using in-memory results for evaluation.")
    loaded_results_for_eval = experiment_all_query_results
elif os.path.exists(EXPERIMENT_RESULTS_FILE):
    print(f"Loading results from '{EXPERIMENT_RESULTS_FILE}' for evaluation...")
    try:
        with open(EXPERIMENT_RESULTS_FILE, 'r') as f:
            loaded_results_for_eval = json.load(f)
        print(f"Successfully loaded {len(loaded_results_for_eval)} results from file.")
    except Exception as e:
        print(f"Error: Could not load results from file. {e}")
else:
    print(f"Error: No results found to evaluate. The variable 'experiment_all_query_results' is not in memory and the file '{EXPERIMENT_RESULTS_FILE}' does not exist.")
# --- END OF CORRECTED LOGIC ---


if loaded_results_for_eval:
    K_VALUES_TO_EVALUATE = [1, 3, 5, 10]
    recall_scores, num_queries = calculate_recall_at_k(loaded_results_for_eval, K_VALUES_TO_EVALUATE)

    print("\n--- Evaluation: Recall@K ---")
    print(f"Evaluated on {num_queries} queries.")
    for k, recall in recall_scores.items():
        print(f"Recall@{k}: {recall:.2f}%")

    # --- Save the full summary to a file ---
    evaluation_summary = {
        "num_queries_evaluated": num_queries,
        "recall_scores_percent": recall_scores,
    }
    with open(EVAL_SUMMARY_SAVE_PATH, 'w') as f_out:
        json.dump(evaluation_summary, f_out, indent=2)
    print(f"\nFull evaluation summary saved to '{EVAL_SUMMARY_SAVE_PATH}'")

    # --- Display Sample Predictions ---
    print("\n--- Sample of Predictions (First 5 Queries) ---")
    for i, res in enumerate(loaded_results_for_eval[:5]):
        true_db = res.get('true_db_id')
        predicted_list = res.get('ranked_predicted_dbs', [])
        is_in_k10 = "Found" if true_db in predicted_list else "Not Found"
        
        print(f"\n{i+1}. Query: '{res.get('nl_query_text', 'N/A')}'")
        print(f"   - True DB:      {true_db} ({is_in_k10} in Top 10)")
        print(f"   - Predicted Top 5: {predicted_list[:5]}")

Using in-memory results for evaluation.

--- Evaluation: Recall@K ---
Evaluated on 830 queries.
Recall@1: 79.40%
Recall@3: 88.92%
Recall@5: 90.60%
Recall@10: 92.05%

Full evaluation summary saved to '/raid/infolab/gaurav/Llama_Spider_A100_Project/experiments_70b_llama/recall_k_results_context_lamma-3.1_8B_CREATE_prompt_all_db_once_top_10_DB_synthetic_examples.json'

--- Sample of Predictions (First 5 Queries) ---

1. Query: 'How many available features are there in total?'
   - True DB:      real_estate_properties (Found in Top 10)
   - Predicted Top 5: ['real_estate_properties', 'product_catalog', 'cre_Doc_Control_Systems', 'theme_gallery', 'cre_Drama_Workshop_Groups']

2. Query: 'What are the name of the countries where there is not a single car maker?'
   - True DB:      car_1 (Found in Top 10)
   - Predicted Top 5: ['car_1', 'manufacturer', 'car_1', 'manufacturer', 'car_1']

3. Query: 'What are the date and the operating professional's first name of each treatment?'
   - True DB:  